<a href="https://colab.research.google.com/github/EnriquezAJP/CCDEPRLR_EXERCISES_COM211/blob/main/Exercise4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 4

In [13]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
import numpy as np
import pandas as pd

dataset = pd.read_json('reviews.json')
#dataset.replace({1:0, 2:0, 3:0, 4:1, 5:1})

## 1. Tokenize the data

In [15]:
sentences = dataset['review'].tolist()
labels = dataset['rating'].tolist()

for x in range(len(labels)):
  if labels[x] <= 3:
    labels[x] = 0
  elif labels[x] >= 4:
    labels[x] = 1

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [16]:
vocab_size = 1000
embedding_dim = 16
max_length = 25
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

## 2. Sequence the data

In [17]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
training_sequences = tokenizer.texts_to_sequences(training_sentences)

## 3. Pad the data

In [18]:
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

## 4. Train a sentiment model

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu6'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 16)            16000     
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 6)                 102       
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 16109 (62.93 KB)
Trainable params: 16109 (62.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
num_epochs = 30
history = model.fit(training_padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/30
25/25 [==============================] - 5s 139ms/step - loss: 0.6901 - accuracy: 0.5638 - val_loss: 0.7124 - val_accuracy: 0.2139
Epoch 2/30
25/25 [==============================] - 3s 101ms/step - loss: 0.6821 - accuracy: 0.5713 - val_loss: 0.7210 - val_accuracy: 0.2090
Epoch 3/30
25/25 [==============================] - 2s 94ms/step - loss: 0.6738 - accuracy: 0.5763 - val_loss: 0.7266 - val_accuracy: 0.2090
Epoch 4/30
25/25 [==============================] - 1s 43ms/step - loss: 0.6639 - accuracy: 0.6037 - val_loss: 0.7233 - val_accuracy: 0.2338
Epoch 5/30
25/25 [==============================] - 1s 38ms/step - loss: 0.6507 - accuracy: 0.6562 - val_loss: 0.7146 - val_accuracy: 0.3184
Epoch 6/30
25/25 [==============================] - 1s 24ms/step - loss: 0.6344 - accuracy: 0.6862 - val_loss: 0.7020 - val_accuracy: 0.3930
Epoch 7/30
25/25 [==============================] - 0s 7ms/step - loss: 0.6143 - accuracy: 0.7412 - val_loss: 0.6789 - val_accuracy: 0.5174
Epoch 8/30
2

## Get files for visualing the network

In [21]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

import io

# Create the reverse word index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

(1000, 16)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 5. Predict sentiment with new reviews

In [22]:
# Use the model to predict a review
fake_reviews = ['Ganda ng shorts', 'Unsatisfied. Poor quality. easily torn.']

print(fake_reviews)

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us

['Ganda ng shorts', 'Unsatisfied. Poor quality. easily torn.']

HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!

1/1 [==============================] - 0s 48ms/step
Ganda ng shorts
[0.9010361]


Unsatisfied. Poor quality. easily torn.
[0.19775727]


